In [1]:
import graphlab

In [3]:
graphlab.SFrame.read_csv?

In [18]:
import sqlite3

conn = sqlite3.connect('/home/sam/data/stormfront/stormfront.sqlite')

In [20]:
author_and_thread = graphlab.SFrame.from_sql(conn, 'SELECT author, thread_url FROM posts')

In [40]:
thread_and_topic_20 = graphlab.SFrame.from_sql(conn, 'SELECT * FROM topics_20')

In [34]:
recommender = graphlab.recommender.item_similarity_recommender.create(
    author_and_thread,
    user_id='author',
    item_id='thread_url',
    similarity_type='jaccard',
    only_top_k=128,
    target_memory_usage=34359738368 # 32GB
)

recommender

Recsys training: model = item_similarity

Preparing data set.

Data has 972846 observations with 15448 users and 66147 items.

Data prepared in: 0.397402s

Training model from provided data.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 16.4129s

Class                            : ItemSimilarityRecommender

Schema
------
User ID                          : author
Item ID                          : thread_url
Target                           : None
Additional observation features  : 0
User side features               : []
Item side features               : []

Statistics
----------
Number of observations           : 972846
Number of users                  : 15448
Number of items                  : 66147

Training summary
----------------
Training time                    : 16.4129

Model Parameters
----------------
Model class                      : ItemSimilarityRecommender
threshold                        : 0.001
similarity_type                  : jaccard
training_method                  : auto

Other Settings
--------------
degree_approximation_threshold   : 4096
sparse_density_estimation_sample_size : 4096
max_data_passes                  : 4096
target_memory_usage              : 34359738368
seed_item_set_size               : 

In [37]:
thread_similarities = recommender.get_similar_items()

thread_similarities

thread_url,similar,score,rank
https://www.stormfront.org/forum/t390760 ...,https://www.stormfront.org/forum/t443722 ...,0.25,1
https://www.stormfront.org/forum/t390760 ...,https://www.stormfront.org/forum/t453305 ...,0.230769217014,2
https://www.stormfront.org/forum/t390760 ...,https://www.stormfront.org/forum/t531807 ...,0.230769217014,3
https://www.stormfront.org/forum/t390760 ...,https://www.stormfront.org/forum/t535995 ...,0.214285731316,4
https://www.stormfront.org/forum/t390760 ...,https://www.stormfront.org/forum/t535327 ...,0.214285731316,5
https://www.stormfront.org/forum/t390760 ...,https://www.stormfront.org/forum/t431257 ...,0.214285731316,6
https://www.stormfront.org/forum/t390760 ...,https://www.stormfront.org/forum/t432921 ...,0.199999988079,7
https://www.stormfront.org/forum/t390760 ...,https://www.stormfront.org/forum/t531784 ...,0.199999988079,8
https://www.stormfront.org/forum/t390760 ...,https://www.stormfront.org/forum/t558455 ...,0.199999988079,9
https://www.stormfront.org/forum/t390760 ...,https://www.stormfront.org/forum/t571291 ...,0.199999988079,10


In [42]:
thread_similarity_by_authors = graphlab.SGraph(vertices=thread_and_topic_20, edges=thread_similarities, vid_field='thread_url', src_field='thread_url', dst_field='similar')

In [43]:
thread_similarity_by_authors

SGraph({'num_edges': 661461, 'num_vertices': 66147})
Vertex Fields:['__id', 'index', 'topic_0', 'topic_1', 'topic_2', 'topic_3', 'topic_4', 'topic_5', 'topic_6', 'topic_7', 'topic_8', 'topic_9', 'topic_10', 'topic_11', 'topic_12', 'topic_13', 'topic_14', 'topic_15', 'topic_16', 'topic_17', 'topic_18', 'topic_19']
Edge Fields:['__src_id', '__dst_id', 'score', 'rank']